# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [119]:
import requests
import re
import os
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [117]:
input_keyword = "獅子"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果
# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

獅子: %E7%8D%85%E5%AD%90
/wiki/%E7%8D%85%E5%AD%90


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [66]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


網路爬蟲（英語：web crawler），也叫網路蜘蛛（spider），是一種用來自動瀏覽全球資訊網的網路機器人。其目的一般為編纂網路索引（英語：Web indexing）。

網路搜尋引擎等站點通過爬蟲軟體更新自身的網站內容（英語：Web content）或其對其他網站的索引。網路爬蟲可以將自己所存取的頁面儲存下來，以便搜尋引擎事後生成索引（英語：Index (search engine)）供用戶搜尋。

爬蟲存取網站的過程會消耗目標系統資源。不少網路系統並不默許爬蟲工作。因此在存取大量頁面時，爬蟲需要考慮到規劃、負載，還需要講「禮貌」。 不願意被爬蟲存取、被爬蟲主人知曉的公開站點可以使用robots.txt檔案之類的方法避免存取。這個檔案可以要求機器人（英語：Software agent）只對網站的一部分進行索引，或完全不作處理。

網際網路上的頁面極多，即使是最大的爬蟲系統也無法做出完整的索引。因此在公元2000年之前的全球資訊網出現初期，搜尋引擎經常找不到多少相關結果。現在的搜尋引擎在這方面已經進步很多，能夠即刻給出高品質結果。

爬蟲還可以驗證超連結和HTML代碼，用於網路抓取（英語：Web scraping）（參見資料驅動編程（英語：Data-driven programming））。

網路爬蟲也可稱作網路蜘蛛[1]、螞蟻、自動索引程式（automatic indexer）[2] ，或（在FOAF（英語：FOAF (software)）軟體中）稱為網路疾走（web scutter）。[3]

網路爬蟲始於一張被稱作種子的統一資源位址（URL）列表。當網路爬蟲存取這些統一資源定位器時，它們會甄別出頁面上所有的超連結，並將它們寫入一張「待訪列表」，即所謂爬行疆域（英語：crawl frontier）。此疆域上的URL將會被按照一套策略迴圈來存取。如果爬蟲在執行的過程中複製歸檔和儲存網站上的資訊，這些檔案通常儲存，使他們可以較容易的被檢視。閱讀和瀏覽他們儲存的網站上並即時更新的資訊，這些被儲存的網頁又被稱為「快照」。越大容量的網頁意味著網路爬蟲只能在給予的時間內下載越少部分的網頁，所以要優先考慮其下載。高變化率意味著網頁可能已經被更新或者被取代。一些伺服器端軟體生成的URL（統一資源定位符）也使得網路爬蟲很難避免檢索到重複內容。

但是網際網路的資源卷帙浩繁

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [100]:
def getExtraUrl(content):
    dictTmp = ({})
    for ext_link in content:
        a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
        if len(a_tag) > 0:
            for link_string in a_tag:
                a_link = link_string["href"]       # 外部連結的網址
                a_keyword = link_string.get_text()  # 外部連結的中文名稱
                tmp={};
                tmp={a_keyword:a_link}
                dictTmp.update(tmp);
    return dictTmp;

In [101]:

getExtraUrl(content)

{'全球資訊網': '/wiki/%E4%B8%87%E7%BB%B4%E7%BD%91',
 '網路機器人': '/wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA',
 '網路': '/wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E',
 '搜尋引擎': '/wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E',
 'robots.txt': '/wiki/Robots.txt',
 '網站': '/wiki/%E7%BD%91%E7%AB%99',
 '超連結': '/wiki/%E8%B6%85%E9%80%A3%E7%B5%90',
 'HTML': '/wiki/HTML',
 '網頁': '/wiki/%E7%B6%B2%E9%A0%81',
 '網際網路': '/wiki/%E4%BA%92%E8%81%94%E7%BD%91',
 '伺服器': '/wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8',
 '超文字傳輸協定': '/wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A'}

## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [120]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        tmp='';
        for line in content:
            tmp+=line.get_text()+'\n';
        filePath='./WikiArticle/'+key_word+".txt";
        if os.path.isfile(filePath):
            print(filePath,'is exist')
        else:
            f = open('./WikiArticle/'+key_word+".txt", "x")
            f.write(tmp)
            f.close()

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = getExtraUrl(content)


                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(v, k, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [121]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E7%8D%85%E5%AD%90 (獅子)
./WikiArticle/獅子.txt is exist
遞迴層[1] - /wiki/%E7%8B%BB%E7%8C%8A (狻猊)
./WikiArticle/狻猊.txt is exist
遞迴層[2] - /wiki/%E4%B8%AD%E5%9C%8B%E7%A5%9E%E8%A9%B1 (中國神話)
./WikiArticle/中國神話.txt is exist
遞迴層[2] - /wiki/%E9%BA%92%E9%BA%9F (麒麟)
./WikiArticle/麒麟.txt is exist
遞迴層[2] - /wiki/%E9%BE%8D (龍)
./WikiArticle/龍.txt is exist
遞迴層[1] - /wiki/%E9%9D%9E%E6%B4%B2 (非洲)
./WikiArticle/非洲.txt is exist
遞迴層[2] - /wiki/%E8%81%AF%E5%90%88%E5%9C%8B%E6%9C%83%E5%93%A1%E5%9C%8B%E5%88%97%E8%A1%A8 (54)
./WikiArticle/54.txt is exist
遞迴層[2] - /wiki/%E6%8B%89%E4%B8%81%E8%AF%AD (拉丁語)
./WikiArticle/拉丁語.txt is exist
遞迴層[2] - /wiki/%E5%9C%B0%E7%90%83 (地球)
./WikiArticle/地球.txt is exist
遞迴層[2] - /wiki/%E6%AC%A7%E6%B4%B2 (歐洲)
./WikiArticle/歐洲.txt is exist
遞迴層[2] - /wiki/%E4%BA%9A%E6%B4%B2 (亞洲)
./WikiArticle/亞洲.txt is exist
遞迴層[2] - /wiki/%E8%B5%A4%E9%81%93 (赤道)
./WikiArticle/赤道.txt is exist
遞迴層[2] - /wiki/%E9%9D%A2%E7%A9%8D (面積)
./WikiArticle/面積.txt is exist
遞迴層[2] - /wiki/%E5%B9%B3%E6

遞迴層[2] - /wiki/%E5%8A%A0%E5%B0%94%E6%96%87%E5%AE%97 (加爾文宗)
遞迴層[2] - /wiki/%E8%B1%AA%E8%90%A8%E4%BA%BA (豪薩人)
遞迴層[2] - /wiki/%E5%9C%A3%E6%88%98 (聖戰)
遞迴層[2] - /wiki/%E5%8E%84%E7%AB%8B%E7%89%B9%E9%87%8C%E4%BA%9A (厄利垂亞)
遞迴層[2] - /wiki/%E5%87%A0%E5%86%85%E4%BA%9A (幾內亞)
遞迴層[2] - /wiki/%E5%90%89%E5%B8%83%E6%8F%90 (吉布地)
遞迴層[2] - /wiki/%E7%A7%91%E6%91%A9%E7%BD%97 (葛摩)
遞迴層[2] - /wiki/%E6%A1%91%E5%90%89%E5%B7%B4%E5%B0%94 (桑吉巴爾)
遞迴層[2] - /wiki/%E8%B5%A4%E9%81%93%E5%B9%BE%E5%85%A7%E4%BA%9E (赤道幾內亞)
遞迴層[2] - /wiki/%E4%B9%8C%E5%B9%B2%E8%BE%BE (烏干達)
遞迴層[2] - /wiki/%E5%8D%9A%E8%8C%A8%E7%93%A6%E7%B4%8D (波札那)
遞迴層[2] - /wiki/%E8%8E%B1%E7%B4%A2%E6%89%98 (賴索托)
遞迴層[2] - /wiki/%E8%8B%B1%E5%9B%BD (英國)
遞迴層[2] - /wiki/%E6%AF%94%E5%88%A9%E6%97%B6 (比利時)
遞迴層[2] - /wiki/%E5%B0%BC%E6%97%A5%E5%88%A9%E4%BA%9A (奈及利亞)
遞迴層[2] - /wiki/%E8%82%AF%E5%B0%BC%E4%BA%9A (肯亞)
遞迴層[2] - /wiki/%E5%9D%A6%E6%A1%91%E5%B0%BC%E4%BA%9A (坦尚尼亞)
遞迴層[2] - /wiki/%E5%8D%9A%E7%A7%91%E5%9C%A3%E5%9C%B0 (博科聖地)
遞迴層[2] - /wiki/%E7%B4%A2%E9%A9%AC%E9%87%8C

KeyboardInterrupt: 